# Homework Lecture 2: Linear Regression

## Preliminaries

### Imports

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.optimize
import sklearn.datasets
import sklearn.linear_model


%matplotlib inline



### Data Directories 

Create a directory with the path below

In [2]:
raw_data_dir="../../raw/california_housing"
data_dir="../../data/probabilisticTools"


### Random Seed

In [3]:
seed=2506
np.random.seed(seed)

### Get Data

<div class="alert alert-block alert-success"> Problem 0 </div>
We download the California housing dataset using the function `sklearn.datasets.fetch_california_housing`.

In [4]:
import sklearn.datasets
housing=sklearn.datasets.fetch_california_housing()

In [5]:
housing.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR'])

In [6]:
print(housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [7]:
print(len(housing.feature_names),housing.feature_names)

8 ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [8]:
print(housing.data.shape,housing.target.shape)

(20640, 8) (20640,)


In [9]:
data=pd.DataFrame(housing.data,columns=housing.feature_names)
data["value"]=housing.target
data.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,value
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


## Data Pre-Processing

The variables in `data` have very different scales.
We will replace the values  $x$ on each column by their standarized values 
$$
    z = \frac{x - \bar{x}}{\sigma_x}
$$

<div class="alert alert-block alert-info"> Problem 1.1 </div>
Compute the mean and std deviation of each column in `data`

[HINT] Pandas has convenient functions to compute the column mean an std deviation

In [10]:
mean = data.mean()
mean

MedInc           3.870671
HouseAge        28.639486
AveRooms         5.429000
AveBedrms        1.096675
Population    1425.476744
AveOccup         3.070655
Latitude        35.631861
Longitude     -119.569704
value            2.068558
dtype: float64

In [11]:
std = data.std()
std

MedInc           1.899822
HouseAge        12.585558
AveRooms         2.474173
AveBedrms        0.473911
Population    1132.462122
AveOccup        10.386050
Latitude         2.135952
Longitude        2.003532
value            1.153956
dtype: float64

<div class="alert alert-block alert-info"> Problem 1.2 </div>
Create a new `DataFrame` called `data_standarized` the value $x$ of each column gets replaced by its standarized value 
$$
    z = \frac{x - \bar{x}}{\sigma_x}
$$

In [12]:
std_value = (data - mean) / std
data_standarized = pd.DataFrame(std_value,columns = housing.feature_names + ['value'])
data_standarized

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,value
0,2.344709,0.982119,0.628544,-0.153754,-0.974405,-0.049595,1.052523,-1.327803,2.129580
1,2.332181,-0.607004,0.327033,-0.263329,0.861418,-0.092510,1.043159,-1.322812,1.314124
2,1.782656,1.856137,1.155592,-0.049015,-0.820757,-0.025842,1.038478,-1.332794,1.258663
3,0.932945,1.856137,0.156962,-0.049832,-0.766010,-0.050328,1.038478,-1.337785,1.165072
4,-0.012881,1.856137,0.344702,-0.032905,-0.759828,-0.085614,1.038478,-1.337785,1.172871
5,0.087445,1.856137,-0.269723,0.014669,-0.894049,-0.089616,1.038478,-1.337785,0.544598
6,-0.111364,1.856137,-0.200913,-0.306626,-0.292704,-0.090723,1.033796,-1.337785,0.800240
7,-0.395127,1.856137,-0.255226,-0.073540,-0.237073,-0.123473,1.033796,-1.337785,0.299354
8,-0.942336,1.061575,-0.458691,0.044253,-0.193805,-0.100497,1.033796,-1.342777,0.171967
9,-0.094467,1.856137,-0.185279,-0.224682,0.110841,-0.086499,1.033796,-1.337785,0.470071


<div class="alert alert-block alert-info"> Problem 1.3</div>
1. Create a numpy array variable named `X` with all the features (but excluding the house values)
2. Create a numpy array variable named `Y` with the house prices (values)

In [13]:
X = data_standarized[housing.feature_names].as_matrix()#creating a numpy array variable
X

/Users/zhouzihui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[ 2.34470896,  0.98211887,  0.62854423, ..., -0.04959533,
         1.05252278, -1.32780305],
       [ 2.33218146, -0.60700421,  0.32703343, ..., -0.09250999,
         1.04315928, -1.32281187],
       [ 1.78265622,  1.85613656,  1.15559247, ..., -0.0258419 ,
         1.03847753, -1.33279424],
       ...,
       [-1.14256563, -0.92482882, -0.09031584, ..., -0.07173277,
         1.77819439, -0.82369324],
       [-1.05455737, -0.84537267, -0.04021014, ..., -0.09122294,
         1.77819439, -0.87360511],
       [-0.78011057, -1.00428498, -0.07044081, ..., -0.0436811 ,
         1.75010387, -0.83367562]])

In [14]:
Y = data_standarized['value'].as_matrix()
Y

/Users/zhouzihui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([ 2.12957989,  1.3141243 ,  1.25866292, ..., -0.99272244,
       -1.05858282, -1.01785337])

## Exact Solution with Numpy

We assume a linear model
$$
     y = \sum_d x_d \theta_d  + \epsilon
$$
where $d$ runs through the housing features and $\epsilon$ is a Gaussian noise term.

<div class="alert alert-block alert-info"> Problem 2.1 </div>
Can you find a reason why we have not included a bias term `b` in the equation?

In [15]:
# Because we've already changed the variable x to its standarized form in the above steps, therefore b will equal to zero.

<div class="alert alert-block alert-info"> Problem 2.2 </div>
Using only `numpy` matrix algebra functions, find the Maximum Likelihood values of $\theta_d$

[Hint] Computing matrix inverses is computationally expensive.  The function `numpy.linalg.solve` can be used to solve systems of linear equations.

In [16]:
ml_theta = np.linalg.solve(np.dot(X.T,X),np.dot(X.T,Y))
ml_theta

array([ 0.71895227,  0.10291078, -0.23010693,  0.26491789, -0.00390232,
       -0.03408034, -0.77984545, -0.75441522])

<div class="alert alert-block alert-info"> Problem 2.3 </div>
Create a variable named `Y_pred` that for each sample $X$, constains  the maximum likelihood model predicted value for $Y$

In [17]:
Y_pred = np.dot(X,ml_theta)
Y_pred

array([ 1.78784232,  1.65348419,  1.39347822, ..., -1.64417577,
       -1.51604801, -1.34559232])

## Gradient Descent Optimization

We will now solve the same problem using Gradient Descent, instead of the analytic solution.

<div class="alert alert-block alert-info"> Problem 3.1 </div>
Define a python function `mse(theta,X,Y)` that computes the mean square error function given $\theta$, $X$ and $Y$

In [18]:
def mse(theta,X,Y):
    se = np.square(np.dot(X,theta)-Y)
    mse = np.mean(se)/2  
    return mse

<div class="alert alert-block alert-info"> Problem 3.2 </div>
Define a python function `grad(theta,X,Y)` that computes the gradient of the error function given $\theta$, $X$ and $Y$

In [19]:
def grad(theta,X,Y): 
    m = len(X)     #len(X) returns the number of rows in the matrix X
    grad = np.dot(X.T,np.dot(X,theta)-Y)/m
    return grad

<div class="alert alert-block alert-info"> Problem 3.3 </div>
Using [`numpy.random.normal`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.normal.html) 
generate a random guess of the vector $\theta$ so that each component is $\mathcal{N}(0,1)$ distributed

In [20]:
mu, sigma = 0, 1
rg_theta = np.random.normal(mu, sigma, X.shape[1])
rg_theta

array([ 1.18332114,  1.3144417 , -1.43101293, -0.90304957,  1.47576176,
       -1.11638228,  1.71375566, -0.16266391])

<div class="alert alert-block alert-info"> Problem 3.4 </div>
Use the function [`scipy.optimze.check_grad`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.check_grad.html)
to verify numerically that `grad` is really the gradient of `mse` for the  $\theta$ guess.

[HINT] `grad` is the gradient of `mse` if `check_grad` returns a very small number (say $\approx 10^-8$)

In [21]:
from scipy.optimize import check_grad
scipy.optimize.check_grad(mse, grad, rg_theta, X, Y) #since it returns a very small number --> grad is the gradient

2.5193743230817395e-07

<div class="alert alert-block alert-info"> Problem 3.5 </div>
** Steepest Descent Algorithm**

1. Pick a value for the learning rate $\eta$
1. Implement the steepest descent update rule
    $$
        \theta \leftarrow \theta - \eta \frac{\partial E}{\partial \theta}
    $$
1. Run the update rule on a loop, starting from your random guess for $\theta$. Repeat  $T=1000$ times
1. Every 100 steps, print the step number and the current error
1. After 1000 steps, print the final error, and the final $\theta$ parameters.
2. If process did not converge, modify value of learning rate $\eta$ and repeat until convergence.

In [22]:
n = 0.5
T = 1000
theta = rg_theta
for t in range(T):
    theta = theta - n*grad(theta, X, Y)
    # then print the step number and the current error for each 100 steps
    if (t % 100 == 0):
        print('step number:', t, 'the current error:', mse(theta, X, Y))   
print('the final error is after', T, 'times is', mse(theta, X, Y))
print('theta:', theta)
# the first try: n = 1, the process did not converge
# the nchanged n = 0.5, it converges obviously

step number: 0 the current error: 0.9737484582556808
step number: 100 the current error: 0.19775497424567143
step number: 200 the current error: 0.19688270264246163
step number: 300 the current error: 0.19687420376097656
step number: 400 the current error: 0.1968741193111804
step number: 500 the current error: 0.19687411847163477
step number: 600 the current error: 0.19687411846328848
step number: 700 the current error: 0.19687411846320552
step number: 800 the current error: 0.19687411846320466
step number: 900 the current error: 0.19687411846320466
the final error is after 1000 times is 0.19687411846320463
theta: [ 0.71895227  0.10291078 -0.23010693  0.26491789 -0.00390232 -0.03408034
 -0.77984545 -0.75441522]


<div class="alert alert-block alert-info"> Problem 3.6 </div>
Compare the MSE of the steepest descent solution to the exact solution.

In [23]:
st_sol = mse(theta, X, Y)
exact_sol = mse(ml_theta, X, Y)
diff = st_sol - exact_sol
print('the steepest descent solution:', st_sol)
print('the exact descent solution:', exact_sol)
print('the difference of the two solutions is:', diff)

the steepest descent solution: 0.19687411846320463
the exact descent solution: 0.19687411846320466
the difference of the two solutions is: -2.7755575615628914e-17


<div class="alert alert-block alert-info"> Problem 3.7 </div>
Compare the  steepest descent parameters $\theta$  to the exact solution.

In [24]:
theta_diff = theta - ml_theta
print('      ', 'steepest𝜃        ' , 'exact solution', '      difference')
for i in range(X.shape[1]):
    print(data.columns[i], theta[i], ml_theta[i], theta_diff[i]) # X was set to array in the above steps

       steepest𝜃         exact solution       difference
MedInc 0.7189522722724133 0.7189522722254268 4.698652578127849e-11
HouseAge 0.1029107797229423 0.10291077971424403 8.698264331030714e-12
AveRooms -0.23010693271850396 -0.23010693262952092 -8.898304315607675e-11
AveBedrms 0.26491789421449813 0.264917894140825 7.367312315764707e-11
Population -0.0039023236401504167 -0.003902323642704787 2.5543703437158616e-12
AveOccup -0.03408034125730446 -0.03408034125556681 -1.737651689204256e-12
Latitude -0.7798454454435649 -0.7798454455510937 1.0752876367092767e-10
Longitude -0.7544152219949256 -0.7544152220969756 1.0205003508900745e-10


## Sklearn Comparison

<div class="alert alert-block alert-info"> Problem 4.1 </div>
Use [`sklearn.linear_model.LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
to fit our model.

[Hint] You will need to create a `LinearRegression` object, and the call the `fit` method. Make sure not to fit the intercept (bias).


In [25]:
LinearRegression = sklearn.linear_model.LinearRegression(fit_intercept = False)
LinearRegression = LinearRegression.fit(X,Y)
LinearRegression

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

<div class="alert alert-block alert-info"> Problem 4.2 </div>
Compute the mean squared different between the exact model prediction's  `Y_pred`  we saved before and
`sklearn`'s Linear model regression predictions

In [26]:
def msd(X,Y_pred,LinearRegression):
    Difference = LinearRegression.predict(X) - Y_pred
    length = len(Y_pred)
    return np.dot(Difference.T,Difference)/length
msd = msd(X,Y_pred,LinearRegression)
msd

3.9626724673140836e-30

<div class="alert alert-block alert-info"> Problem 4.3 </div>
Compare the sklearn solution to the exact solution we found earlier.

[Hint] The solution is stored on the model's  `coef_` variable

In [27]:
print('    ', '     exact solution', '   sklearn solution')
for i in range(X.shape[1]):
    print(data.columns[i], ml_theta[i], LinearRegression.coef_[i])

          exact solution    sklearn solution
MedInc 0.7189522722254268 0.7189522722254276
HouseAge 0.10291077971424403 0.10291077971424487
AveRooms -0.23010693262952092 -0.23010693262952173
AveBedrms 0.264917894140825 0.2649178941408253
Population -0.003902323642704787 -0.0039023236427050455
AveOccup -0.03408034125556681 -0.034080341255566575
Latitude -0.7798454455510937 -0.779845445551089
Longitude -0.7544152220969756 -0.7544152220969702


### Statmodels  Comparison

In [28]:
import statsmodels.api as sm

We will solve using  `statmodels` so that we appreciate the difference in emphasis between Machine Learning (`sklearn`) and Statistics Modeling `statmodels` 

<div class="alert alert-block alert-info"> Problem 5.1 </div>
Use [`statmodels.api.OLS`](http://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html) to solve the same linear regression problem


In [29]:
stats_lr = sm.OLS(Y,X)
result = stats_lr.fit()

<div class="alert alert-block alert-info"> Problem 5.2 </div>
Compare the `statmodels` solution to the exact solution we found earlier.

[Hint] The fitted parameters are stored on the results 's  `params` variable

In [30]:
print('    ', '     exact solution', '   statmodel solution')
for i in range(X.shape[1]):
    print(data.columns[i], ml_theta[i], result.params[i])

          exact solution    statmodel solution
MedInc 0.7189522722254268 0.7189522722254271
HouseAge 0.10291077971424403 0.1029107797142449
AveRooms -0.23010693262952092 -0.23010693262952087
AveBedrms 0.264917894140825 0.2649178941408249
Population -0.003902323642704787 -0.003902323642705284
AveOccup -0.03408034125556681 -0.03408034125556665
Latitude -0.7798454455510937 -0.7798454455510893
Longitude -0.7544152220969756 -0.7544152220969701


<div class="alert alert-block alert-info"> Problem 5.3 </div>
Print a  `statmodels` result summary (function `summary` of the results object).

It will show you a number of estimates on goodness-of-fit, significance of coefficients, etc.

In [31]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.606
Model:                            OLS   Adj. R-squared (uncentered):              0.606
Method:                 Least Squares   F-statistic:                              3971.
Date:                Fri, 20 Sep 2019   Prob (F-statistic):                        0.00
Time:                        00:24:41   Log-Likelihood:                         -19668.
No. Observations:               20640   AIC:                                  3.935e+04
Df Residuals:                   20632   BIC:                                  3.942e+04
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.7190      0.007    104.056      0.000       0.705       0.732
x2             0.1029      0.005     21.144      0.000       0.093       0.112
x3            -0.2301      0.013    -18.236      0.000      -0.255      -0.205
x4             0.2649      0.012     22.928      0.000       0.242       0.288
x5            -0.0039      0.005     -0.837      0.402      -0.013       0.005
x6            -0.0341      0.004     -7.769      0.000      -0.043      -0.025
x7            -0.7798      0.013    -58.543      0.000      -0.806      -0.754
x8            -0.7544      0.013    -57.684      0.000      -0.780      -0.729
==============================================================================
Omnibus:                     4393.650   Durbin-Watson:                   0.885
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14087.596
Skew:                           1.082   Prob(JB):                         0.00
Kurtosis:                       6.420   Cond. No.                         6.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Independent test for categorial variables

<div class="alert alert-block alert-info"> Problem 6.1 </div>

Read the data from file 'homework.csv' in the  'data_dir' directory (you must run `GenerateCategorialVariables` notebook first to create the data)

Perform a $\chi^2$ test of independence between the variables `X` and `Y`.
Are 'X' and 'Y' dependent on each other?

[Hint] You can copy any code you need from the [`CategoricalInference`](./CategoricalInference.ipynb) Notebook,
but make sure to import any python modules you may need.

In [32]:
import scipy.special as special

# Import the file
base_data_dir="../../data/ProbabilisticTools"
os.chdir(base_data_dir)
df = pd.read_csv('homework.csv')

# Create the hot-noded matrices
X = df["X"]
Y = df["Y"]
X_hn = pd.get_dummies(X).as_matrix()
Y_hn = pd.get_dummies(Y).as_matrix()

# Define a chi_test function
def C2_independence(Z_x,Z_y):
    N=len(Z_x)
    D=Z_x.shape[1]
    K=Z_y.shape[1]
    # p_y has index k
    p_y=Z_y.mean(axis=0)
    # p_x has index d
    p_x=Z_x.mean(axis=0)
    # p will be K*D, with indexes k,d
    p=p_y[:,np.newaxis]*p_x[np.newaxis,:]
    # expectation if x and y are independent
    expect=N*p
    # Z_y has indexes i,k and Z_x has indexes i,d
    #Z will be N*K*D, with indexes i,k,d
    Z=Z_y[:,:,np.newaxis]*Z_x[:,np.newaxis,:]
    # observations for each (y,x) 
    # sum over i, left with a K*D matrix
    obs=Z.sum(axis=0) # last two expressions are the same as np.dot(Z_y^T,Z_x)
    df=obs-expect
    df2=df*df
    # we need to special case 0/0 case.
    c2 = (df2/np.maximum(1e-9,expect)).sum()
    return c2,special.chdtrc((K-1)*(D-1),c2)

a = C2_independence(X_hn,Y_hn)

/Users/zhouzihui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/Users/zhouzihui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [33]:
print ('The probability that they are independent is:', a[1]*100, '%. Therefore, we accept them as independent.')

The probability that they are independent is: 6.1498086442245095 %. Therefore, we accept them as independent.
